In this ipynb I need to check what data I already received from my scrapes by the accounts or researcherIds I received from the parenthood survey. First, I load the 1220 researchers received by ORCIDS (some of them were received through given ScopusIds) which were then scraped from Open Alex. 
Second, I used SerpAPI to scrape publication data of researchers by their GoogleScholar URL received from the survey.

# Open Alex Data

In [2]:
import pandas as pd

file_path = "openAlex_scrapes_with_orcids/orcid_and_scopus_from_oA_combined.pkl"
orcid_scopus_df = pd.read_pickle(file_path)

orcid_scopus_df.head()

,ResponseId,works_count,cited_by_count,counts_by_year,affiliations,last_known_institutions,works_api_url,main_domain
0,R_2DUIfkur5fbOgZo,132,1211,"[{'year': 2025, 'works_count': 3, 'cited_by_co...",[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I102134673', 'ro...",https://api.openalex.org/works?filter=author.i...,Social Sciences
1,R_46yreKgPLBawn3H,55,813,"[{'year': 2025, 'works_count': 0, 'cited_by_co...",[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I91136226', 'ror...",https://api.openalex.org/works?filter=author.i...,Physical Sciences
2,R_2ZTGtufLu3KUHW9,30,224,"[{'year': 2025, 'works_count': 7, 'cited_by_co...",[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I322491407', 'ro...",https://api.openalex.org/works?filter=author.i...,Health Sciences
3,R_1gzm4gKhyxKNWBW,11,24,"[{'year': 2025, 'works_count': 1, 'cited_by_co...",[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I8765205', 'ror'...",https://api.openalex.org/works?filter=author.i...,Physical Sciences
4,R_3w5HE5P6WKA3ErS,14,24,"[{'year': 2025, 'works_count': 1, 'cited_by_co...",[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210095952', 'r...",https://api.openalex.org/works?filter=author.i...,Social Sciences


In [3]:
len(orcid_scopus_df)

1220

In [4]:
# recheck if there are any NaNs in any row

orcid_scopus_df.isnull().any(axis=1).any()

np.False_

# SerpAPI Data

In [131]:
serpapi_file_path = "serpAPI_scrapes/main_scrapes/gs_df_seventh_scrape.xlsx"
googleScholar_df = pd.read_excel(serpapi_file_path)
googleScholar_df.head()

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
0,R_4pD7OrlAt0l6mjL,https://scholar.google.com/citations?user=Guhy...,GuhyORoAAAAJ,{'search_metadata': {'id': '68703f16634a638ac3...,2.0,NaN,NaN,94.0,NaN,NaN,NaN,NaN
1,R_7OE9SfkJJ6q9jbP,https://scholar.google.com/citations?user=BSke...,BSkeONEAAAAJ,{'search_metadata': {'id': '68703f17874d50617c...,0.0,NaN,NaN,NaN,[{'title': 'GDPR Interference With Next Genera...,"{'table': [{'citations': {'all': 45, 'since_20...","[{'year': 2020, 'citations': 3}, {'year': 2021...","{'name': 'Stavroula Rizou, Ph.D.', 'affiliatio..."
2,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...,NaN,NaN,2.0,1.0,1.0,187.0,NaN,NaN,NaN,NaN
3,R_2B4eUokwACUsT08,https://scholar.google.com/citations?user=DCWk...,DCWkn9sAAAAJ,{'search_metadata': {'id': '68703f19abe7eca231...,1.0,1.0,NaN,118.0,[{'title': 'Ecological impact of introduced fi...,"{'table': [{'citations': {'all': 1169, 'since_...","[{'year': 2011, 'citations': 7}, {'year': 2012...","{'name': 'Rocco Tiberti', 'affiliations': 'DiB..."
4,R_2JTY94aJonph8Qx,https://scholar.google.it/citations?user=Uyrfi...,UyrfibgAAAAJ,{'search_metadata': {'id': '68703f1b95e95031e3...,1.0,1.0,NaN,118.0,[{'title': 'Agent based modeling and simulatio...,"{'table': [{'citations': {'all': 4117, 'since_...","[{'year': 2004, 'citations': 28}, {'year': 200...","{'name': 'Giuseppe Vizzari', 'affiliations': '..."


In [136]:
# I should have used jason files

serpapi_file_path = "serpAPI_scrapes/main_scrapes/gs_df_seventh_scrape.json"
googleScholar_df = pd.read_json(serpapi_file_path, lines=True)

In [137]:
len(googleScholar_df)

928

# Preprocessing

In [138]:
# How many overlapping ResponseIds/ Individuals from the survey provided both: ORCID and Google Scholar Account

common_response_ids = set(googleScholar_df['ResponseId']).intersection(orcid_scopus_df['ResponseId'])
print(f"Number of common ResponseIds: {len(common_response_ids)}")

Number of common ResponseIds: 382


In [ ]:
import json

def raw_author_data_is_error(val):
    if pd.isna(val):
        return False
    if isinstance(val, dict):
        return "error" in json.dumps(val).lower()
    if isinstance(val, str):
        return "error" in val.lower()
    return False


def other_fields_empty(row):
    for col in ['full_articles', 'cited_by', 'citation_graph', 'author_metadata']:
        val = row.get(col)

        if val is None:
            continue
        if isinstance(val, (list, dict)) and len(val) == 0:
            continue
        if isinstance(val, str) and val.strip() in ["", "[]", "{}", "nan"]:
            continue

        return False  # found non-empty
    return True  # all are empty


def row_matches(row):
    return raw_author_data_is_error(row['raw_author_data']) and other_fields_empty(row)

mask = googleScholar_df.apply(row_matches, axis=1)

print(f"Rows where raw_author_data has 'error' AND other fields are empty: {mask.sum()}")


Rows where raw_author_data has 'error' AND other fields are empty: 148


In [144]:
error_df = googleScholar_df[mask]
display(error_df)

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
105,R_2Yrpffz60uwjaXO,https://scholar.google.com/citations?user=P5kx...,P5kxXSUAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,187.0,None,None,None,None
111,R_2CTHs0Ex7u8ypx2,https://scholar.google.com/citations?hl=en&use...,Qz5gcqIAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,243.0,None,None,None,None
116,R_2sX9dwhhcZA3F8I,https://scholar.google.pl/citations?user=lgADk...,lgADkJQAAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,187.0,None,None,None,None
119,R_2NriW7rxL5uNUAU,https://scholar.google.co.uk/citations?user=7d...,7dh2dL0AAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,63.0,None,None,None,None
124,R_2llWAOpDuSAyB3P,https://scholar.google.com/citations?user=mGxG...,mGxGxZcAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,94.0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
909,R_2V8YuyOjKZTO9Dj,https://scholar.google.be/citations?user=qj_0W...,qj_0W3wAAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,166.0,None,None,None,None
912,R_8xOS5xGxK3T62EE,https://scholar.google.com/citations?user=cEZu...,cEZuEmoAAAAJ,{'error': 'Your account has run out of searche...,NaN,NaN,NaN,NaN,None,None,None,None
914,R_2roxXEDlaPt6Tt9,https://0-scholar-google-com.brum.beds.ac.uk/c...,6ZMavB4AAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,NaN,None,None,None,None
916,R_4hwFaE2woHHkzXb,https://scholar.google.com/citations?user=FoAg...,FoAgYl0AAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,236.0,None,None,None,None


In [145]:
problem_rows = googleScholar_df[googleScholar_df.apply(row_matches, axis=1)]

problem_response_ids = set(problem_rows['ResponseId'])

# Compute intersection with the known OpenAlex response IDs
num_common = len(problem_response_ids.intersection(common_response_ids))
print(f"Number of ResponseIds without data avalable in Open Alex data: {num_common}")

Number of ResponseIds without data avalable in Open Alex data: 57


In [ ]:
# I can remove the rows that are covered by OpenAlex Scrape in DoogleScholar_df, but save those that were not retrieved by SerpAPI nor present in OpenAlex for another scrape

gs_scrape_failed = set(error_df['ResponseId']) - common_response_ids
gs_scrape_failed = googleScholar_df[googleScholar_df['ResponseId'].isin(gs_scrape_failed)]
len(gs_scrape_failed)

91

In [146]:
display(gs_scrape_failed.head())

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
111,R_2CTHs0Ex7u8ypx2,https://scholar.google.com/citations?hl=en&use...,Qz5gcqIAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,243.0,NaN,NaN,NaN,NaN
119,R_2NriW7rxL5uNUAU,https://scholar.google.co.uk/citations?user=7d...,7dh2dL0AAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,63.0,NaN,NaN,NaN,NaN
124,R_2llWAOpDuSAyB3P,https://scholar.google.com/citations?user=mGxG...,mGxGxZcAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,94.0,NaN,NaN,NaN,NaN
126,R_8SAiIArDDqGGYve,https://scholar.google.com/citations?user=ckIG...,ckIGdh8AAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,118.0,NaN,NaN,NaN,NaN
136,R_8w6OsHeZQcfhWvE,https://scholar.google.nl/citations?user=Cn4yW...,Cn4yWp4AAAAJ,{'error': 'Your account has run out of searche...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# saving unretrieved/ missing ones (and not covered by OpenAlex either)

gs_scrape_failed.to_json("SerpAPI_Scrapes/rescrape/rescrape_2/ResponseIds_without_results_SerpAPI_Google_scholar.json", 
                         lines=True, 
                         orient="records")

In [163]:
adding_retrieved_failed_scrapes = pd.read_json("SerpAPI_Scrapes/rescrape/rescrape_1/readding_found_gs_df_rescraped.json", lines=True)  # these are manually looked up URLs that were provided mistakenly but were easy to identify/ unambiguous
adding_retrieved_failed_scrapes.head()

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata,manually_added_URL_correctly,ORCID
0,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...,eje6YXEAAAAJ,NaN,2,1.0,1.0,187.0,[{'title': 'Employability and the psychologica...,"{'table': [{'citations': {'all': 2432, 'since_...","[{'year': 1996, 'citations': 6}, {'year': 1997...","{'name': 'Piotr Jędrzejowicz', 'affiliations':...",https://scholar.google.com/citations?hl=en&use...,None
1,R_86fDotargQC1YOh,https://scholar.google.be/scholar?hl=nl&as_sdt...,bnrZwNUAAAAJ,NaN,2,1.0,3.0,24.0,[{'title': 'Perceived discrimination in primar...,"{'table': [{'citations': {'all': 130, 'since_2...","[{'year': 2017, 'citations': 2}, {'year': 2018...","{'name': 'Amelie Van Pottelberge', 'affiliatio...",https://scholar.google.be/citations?user=bnrZw...,None
2,R_84BIAAeGyGEwNcq,https://scholar.google.com/scholar?hl=fr&as_sd...,CiaGmjQAAAAJ,NaN,0,NaN,NaN,NaN,[{'title': 'Training general practitioners in ...,"{'table': [{'citations': {'all': 133, 'since_2...","[{'year': 2020, 'citations': 1}, {'year': 2021...","{'name': 'Evelyne Harkemanne', 'affiliations':...",https://scholar.google.com/citations?user=CiaG...,None
3,R_8IuhasFPlu8kYYV,https://scholar.google.com/scholar?hl=no&as_sd...,None,NaN,2,2.0,NaN,82.0,None,None,None,None,None,https://orcid.org/0000-0002-7400-9791
4,R_4dEt5R2bLqUQcnm,https://scholar.google.com/scholar?hl=tr&as_sd...,SdWwo6IAAAAJ,NaN,2,2.0,2.0,236.0,[{'title': 'Use of three-dimensional medical m...,"{'table': [{'citations': {'all': 517, 'since_2...","[{'year': 2008, 'citations': 5}, {'year': 2009...","{'name': 'Banu Saglam Aydinatay', 'affiliation...",https://scholar.google.com/citations?hl=tr&use...,None


In [162]:
len(adding_retrieved_failed_scrapes)
#len(gs_scrape_failed)

26

In [209]:
adding_retrieved_failed_scrapes_2 = pd.read_json("SerpAPI_Scrapes/rescrape/rescrape_2/second_scrape.json", lines=True) 
adding_retrieved_failed_scrapes_2.head()

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
0,R_2CTHs0Ex7u8ypx2,https://scholar.google.com/citations?hl=en&use...,Qz5gcqIAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,243.0,[{'title': 'Not just autonomy--the principles ...,"{'table': [{'citations': {'all': 10559, 'since...","[{'year': 1997, 'citations': 30}, {'year': 199...","{'name': 'Soren Holm', 'affiliations': 'Profes..."
1,R_2NriW7rxL5uNUAU,https://scholar.google.co.uk/citations?user=7d...,7dh2dL0AAAAJ,{'error': 'Your account has run out of searche...,2.0,NaN,NaN,63.0,"[{'title': 'Heidegger on Technology', 'link': ...","{'table': [{'citations': {'all': 146, 'since_2...","[{'year': 2014, 'citations': 1}, {'year': 2015...","{'name': 'Christos Hadjioannou', 'affiliations..."
2,R_2llWAOpDuSAyB3P,https://scholar.google.com/citations?user=mGxG...,mGxGxZcAAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,94.0,[{'title': 'Unraveling negative biotic interac...,"{'table': [{'citations': {'all': 798, 'since_2...","[{'year': 2014, 'citations': 13}, {'year': 201...","{'name': 'Myrto Tsiknia', 'affiliations': 'Pos..."
3,R_8SAiIArDDqGGYve,https://scholar.google.com/citations?user=ckIG...,ckIGdh8AAAAJ,{'error': 'Your account has run out of searche...,1.0,NaN,NaN,118.0,[{'title': 'Facing loneliness and anxiety duri...,"{'table': [{'citations': {'all': 4396, 'since_...","[{'year': 2015, 'citations': 25}, {'year': 201...","{'name': 'Alessandro Musetti, Phd, PsyD', 'aff..."
4,R_8w6OsHeZQcfhWvE,https://scholar.google.nl/citations?user=Cn4yW...,Cn4yWp4AAAAJ,{'error': 'Your account has run out of searche...,NaN,NaN,NaN,NaN,[{'title': 'Influence of partner diversity on ...,"{'table': [{'citations': {'all': 533, 'since_2...","[{'year': 2001, 'citations': 2}, {'year': 2002...","{'name': 'Ariane von Raesfeld', 'affiliations'..."


In [210]:
ids_rescraped = set(adding_retrieved_failed_scrapes['ResponseId'])
ids_more = set(adding_retrieved_failed_scrapes_2['ResponseId'])

# Intersection: which are in both
common_ids = ids_rescraped.intersection(ids_more)
print(f"Number of common ResponseIds: {len(common_ids)}")

Number of common ResponseIds: 0


In [211]:
cols = ['full_articles','cited_by','citation_graph','author_metadata']

base = googleScholar_df.set_index('ResponseId')

for src in (adding_retrieved_failed_scrapes, adding_retrieved_failed_scrapes_2):
    up = src.set_index('ResponseId')[cols].reindex(base.index)

    for c in cols:
        mask = base[c].isna() & up[c].notna()          # update only where base is NaN and src has data
        base.loc[mask, c] = up.loc[mask, c]

googleScholar_df = base.reset_index()

In [214]:
googleScholar_df.head()

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
0,R_7OE9SfkJJ6q9jbP,https://scholar.google.com/citations?user=BSke...,BSkeONEAAAAJ,{'search_metadata': {'id': '68703f17874d50617c...,0.0,NaN,NaN,NaN,[{'title': 'GDPR Interference With Next Genera...,"{'table': [{'citations': {'all': 45, 'since_20...","[{'year': 2020, 'citations': 3}, {'year': 2021...","{'name': 'Stavroula Rizou, Ph.D.', 'affiliatio..."
1,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...,None,None,2.0,1.0,1.0,187.0,[{'title': 'Employability and the psychologica...,"{'table': [{'citations': {'all': 2432, 'since_...","[{'year': 1996, 'citations': 6}, {'year': 1997...","{'name': 'Piotr Jędrzejowicz', 'affiliations':..."
2,R_2B4eUokwACUsT08,https://scholar.google.com/citations?user=DCWk...,DCWkn9sAAAAJ,{'search_metadata': {'id': '68703f19abe7eca231...,1.0,1.0,NaN,118.0,[{'title': 'Ecological impact of introduced fi...,"{'table': [{'citations': {'all': 1169, 'since_...","[{'year': 2011, 'citations': 7}, {'year': 2012...","{'name': 'Rocco Tiberti', 'affiliations': 'DiB..."
3,R_2JTY94aJonph8Qx,https://scholar.google.it/citations?user=Uyrfi...,UyrfibgAAAAJ,{'search_metadata': {'id': '68703f1b95e95031e3...,1.0,1.0,NaN,118.0,[{'title': 'Agent based modeling and simulatio...,"{'table': [{'citations': {'all': 4117, 'since_...","[{'year': 2004, 'citations': 28}, {'year': 200...","{'name': 'Giuseppe Vizzari', 'affiliations': '..."
4,R_8S1zWLINCbSqU7z,https://scholar.google.com/citations?user=jqBv...,jqBvlqkAAAAJ,{'search_metadata': {'id': '68703f1e6f1e407892...,2.0,3.0,3.0,188.0,[{'title': 'Injectable and biodegradable hydro...,"{'table': [{'citations': {'all': 7777, 'since_...","[{'year': 2006, 'citations': 25}, {'year': 200...","{'name': 'João Rodrigues', 'affiliations': 'CQ..."


In [215]:
# I can create a new df, containing only ResponseIds resulting in some scraped data from SerpAPI

cols = ['raw_author_data', 'full_articles', 'cited_by', 'citation_graph', 'author_metadata']
mask_all_nan = googleScholar_df[cols].isnull().all(axis=1)
print(mask_all_nan.sum())

nan_rows_df = googleScholar_df[mask_all_nan]
display(nan_rows_df)


28


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata
22,R_2gESUa3jSIlreut,https://scholar.google.com/scholar?hl=pl&as_sd...,None,None,1.0,3.0,NaN,187.0,None,None,None,None
132,R_8IuhasFPlu8kYYV,https://scholar.google.com/scholar?hl=no&as_sd...,None,None,2.0,2.0,NaN,82.0,None,None,None,None
204,R_27e32EDeDlr8eyU,https://scholar.google.com/,None,None,2.0,3.0,3.0,243.0,None,None,None,None
224,R_2Eau6BBwERmShZW,https://orcid.org/0000-0002-9925-2173,None,None,0.0,NaN,NaN,NaN,None,None,None,None
259,R_82ms9cdQFpWuwkZ,https://www.bwi.uni-stuttgart.de/institut/team...,None,None,1.0,NaN,NaN,90.0,None,None,None,None
282,R_2EnQS3mqO9PZICY,https://scholar.google.com/scholar?hl=de&as_sd...,None,None,3.0,1.0,1.0,90.0,None,None,None,None
374,R_8bIVRT6jaNhc7Ox,4,None,None,3.0,NaN,NaN,90.0,None,None,None,None
395,R_2WBbA5lTecRmhko,https://scholar.google.com/,None,None,3.0,2.0,2.0,82.0,None,None,None,None
412,R_2UhWPd29jTZYebP,Marcus Clauss,None,None,3.0,NaN,NaN,90.0,None,None,None,None
467,R_82omyzLPNQrS0IG,https://ncl.instructure.com/courses/52235/page...,None,None,2.0,2.0,2.0,243.0,None,None,None,None


The last results show those rows of Google Scholar, that still show empty relevant data cols, probably due to mistakes in the provided URL from respondents.

In [216]:
import ast
from pandas.api.types import is_scalar


# parse helper function
def safe_parse(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception as e:
            print("Failed to parse:", val[:100], "...", f"Error: {e}")
            return val
    return val

# are there any rows where 'raw_author_data' contains 'search_metadata' (with useful data) AND the other conditions for parental leave model are available?

def has_search_metadata_and_all_empty(row):
    val = safe_parse(row['raw_author_data'])
    if not (isinstance(val, dict) and 'search_metadata' in val):
        return False
    for col in ['full_articles', 'cited_by', 'citation_graph', 'author_metadata']:
        if not is_empty(row[col]):
            return False
    return True


# are there any rows where 'raw_author_data' contains 'error' (with NOT useful data) AND the other conditions for parental leave model are available?

def is_empty(val):
    val = safe_parse(val)

    if is_scalar(val) and pd.isna(val):
        return True
    if isinstance(val, str):
        s = val.strip()
        return s in ["", "{}", "[]"]
    if isinstance(val, (list, dict)):
        return len(val) == 0
    return False


def has_error_and_all_empty(row):
    val = safe_parse(row['raw_author_data'])
    if not (isinstance(val, dict) and 'error' in val):
        return False
    for col in ['full_articles', 'cited_by', 'citation_graph', 'author_metadata']:
        if not is_empty(row[col]):
            return False
    return True


In [218]:
valid_mask = googleScholar_df.apply(has_search_metadata_and_all_empty, axis=1) & (
    ((googleScholar_df['DE22'] == 0) & (googleScholar_df['PL6_1'].isna())) |
    ((googleScholar_df['DE22'] > 0) & (googleScholar_df['PL6_1'] > 0))
)

error_mask = googleScholar_df.apply(has_error_and_all_empty, axis=1)


In [219]:
# entries with search metadata and parental leave conditions met but no other data in other relevant cols
filtered_valid_df = googleScholar_df[valid_mask]
print(f"\nShowing {len(filtered_valid_df)} rows with valid search metadata and matching parental leave conditions:")
display(filtered_valid_df)

# entries with error in 'raw_author_data' and other relevant cols empty
filtered_error_df = googleScholar_df[error_mask]
print(f"\nShowing {len(filtered_error_df)} rows with error in raw_author_data and all related fields empty:")
display(filtered_error_df)



Showing 0 rows with valid search metadata and matching parental leave conditions:


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata



Showing 0 rows with error in raw_author_data and all related fields empty:


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata


In [220]:
# entries with error OR search_metadata in col 'raw_author_data' and other relevant cols empty

def is_missing_row(row):
    val = safe_parse(row['raw_author_data'])
    is_error = isinstance(val, dict) and 'error' in val
    is_success = isinstance(val, dict) and 'search_metadata' in val
    
    if not (is_error or is_success):
        return False
    
    for col in ['full_articles', 'cited_by', 'citation_graph', 'author_metadata']:
        if not is_empty(row[col]):
            return False
    
    return True

In [221]:
missing_mask = googleScholar_df.apply(is_missing_row, axis=1)

print("Number of missing rows:", missing_mask.sum())
unretrieved_df = googleScholar_df[missing_mask]
display(unretrieved_df)

Number of missing rows: 0


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata


In [186]:
num_filtered_error_in_missing = filtered_error_df.index.intersection(googleScholar_df[missing_mask].index).size
print(f"Number of rows of filtered_error_df that are in missing_mask: {num_filtered_error_in_missing}")

Number of rows of filtered_error_df that are in missing_mask: 73


All rows of filtered_error_df are in missing_mask. I will save these rows for a possible later scrape if they become important but as far as checks have shown, either:  

- not providing any information on their number of children and whether they took parental leave  
- did not provide a valid URL resulting in a wrong User_Id  
- did not retrieve anything from Google Scholar  

This means we need to subtract those from the 928 - 83 = resulting in 845 Researchers additional to the already available Open Alex ones (approximately 1200)

In [ ]:
unretrieved_df.to_json(
    "not_found_data/unretrieved_GoogleScholar_accounts.json",
    lines=True,
    orient="records"
)

In [192]:
googleScholar_df = googleScholar_df[~missing_mask]
len(googleScholar_df)

/var/folders/4n/x6xrnlyj6nngzc77qzc7h4f40000gn/T/ipykernel_81666/3707980338.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  googleScholar_df = googleScholar_df[~missing_mask]


845

In [ ]:
googleScholar_df.to_json(
    #"valid_GoogleScholar_Results.json", 
    lines=True, 
    orient="records"
)

# Data Harmonization & Integration 

### Data from OpenAlex Scrape

In [226]:
orcid_scopus_df.columns

Index(['ResponseId', 'works_count', 'cited_by_count', 'counts_by_year',
       'affiliations', 'last_known_institutions', 'works_api_url',
       'main_domain'],
      dtype='object')

In [227]:
import json
print(type(orcid_scopus_df['counts_by_year'].iloc[0]))
print(json.dumps(orcid_scopus_df['counts_by_year'].iloc[0], indent=2))


<class 'list'>
[
  {
    "year": 2025,
    "works_count": 3,
    "cited_by_count": 33
  },
  {
    "year": 2024,
    "works_count": 2,
    "cited_by_count": 80
  },
  {
    "year": 2023,
    "works_count": 7,
    "cited_by_count": 78
  },
  {
    "year": 2022,
    "works_count": 1,
    "cited_by_count": 44
  },
  {
    "year": 2021,
    "works_count": 7,
    "cited_by_count": 52
  },
  {
    "year": 2020,
    "works_count": 3,
    "cited_by_count": 56
  },
  {
    "year": 2019,
    "works_count": 12,
    "cited_by_count": 59
  },
  {
    "year": 2018,
    "works_count": 3,
    "cited_by_count": 41
  },
  {
    "year": 2017,
    "works_count": 6,
    "cited_by_count": 57
  },
  {
    "year": 2016,
    "works_count": 13,
    "cited_by_count": 69
  },
  {
    "year": 2015,
    "works_count": 1,
    "cited_by_count": 54
  },
  {
    "year": 2014,
    "works_count": 1,
    "cited_by_count": 38
  },
  {
    "year": 2013,
    "works_count": 2,
    "cited_by_count": 62
  },
  {
    "year": 201

### SerpAPI Scrape

In [ ]:
# reload 

googleScholar_df = pd.read_json("valid_GoogleScholar_Results.json", lines=True)
googleScholar_df.columns

In [201]:
first_entry = googleScholar_df['citation_graph'].iloc[0]
print(json.dumps(first_entry, indent=2))

[
  {
    "year": 2020,
    "citations": 3
  },
  {
    "year": 2021,
    "citations": 7
  },
  {
    "year": 2022,
    "citations": 10
  },
  {
    "year": 2023,
    "citations": 8
  },
  {
    "year": 2024,
    "citations": 10
  },
  {
    "year": 2025,
    "citations": 6
  }
]


In [229]:
first_entry = googleScholar_df['full_articles'].iloc[0]
print(json.dumps(first_entry, indent=2))

[
  {
    "title": "GDPR Interference With Next Generation 5G and IoT Networks",
    "link": "https://scholar.google.com/citations?view_op=view_citation&hl=en&user=BSkeONEAAAAJ&pagesize=100&citation_for_view=BSkeONEAAAAJ:u5HHmVD_uO8C",
    "citation_id": "BSkeONEAAAAJ:u5HHmVD_uO8C",
    "authors": "S Rizou, E Alexandropoulou-Egyptiadou, KE Psannis",
    "publication": "IEEE Access 8, 108052-108061, 2020",
    "cited_by": {
      "value": 30,
      "link": "https://scholar.google.com/scholar?oi=bibs&hl=en&cites=13481622825000470092",
      "serpapi_link": "https://serpapi.com/search.json?cites=13481622825000470092&engine=google_scholar&hl=en",
      "cites_id": "13481622825000470092"
    },
    "year": "2020"
  },
  {
    "title": "Preserving Minors' Data Protection in IoT-based Smart Homes According to GDPR Considering Cross-Border Issues.",
    "link": "https://scholar.google.com/citations?view_op=view_citation&hl=en&user=BSkeONEAAAAJ&pagesize=100&citation_for_view=BSkeONEAAAAJ:u-x6o8

As we have a yearly "works_count" number, its citations until today "cited_by_count" per "year" in OpenAlex data, we want to create the same for the SerpAPI results, and we can get this by extracting these data from the "full_articles" column. Here we need to extract for each title, the "year", the "cited_by":"value" vaiable and then sort how many are in the same year per unique ResponseId. Then we need to create the same structure a list of dicts where each year is its own dict containing, "year", "works_count", "cited_by_count".

In [230]:
print(type(googleScholar_df['full_articles'].iloc[0]))

<class 'list'>


In [231]:
from collections import defaultdict

def extract_counts_by_year(article_list):
    counts = defaultdict(lambda: {"works_count": 0, "cited_by_count": 0})

    for article in article_list:
        year = article.get("year")
        if not year or not str(year).isdigit():
            continue
        year = int(year)

        citations = article.get("cited_by", {}).get("value", 0)
        if citations is None:
            citations = 0

        counts[year]["works_count"] += 1
        counts[year]["cited_by_count"] += citations

    return [
        {"year": year, **counts[year]} 
        for year in sorted(counts.keys())
    ]


In [233]:
sample_articles = googleScholar_df['full_articles'].iloc[0]
result = extract_counts_by_year(sample_articles)
print(json.dumps(result, indent=2))

[
  {
    "year": 2020,
    "works_count": 2,
    "cited_by_count": 32
  },
  {
    "year": 2021,
    "works_count": 2,
    "cited_by_count": 3
  },
  {
    "year": 2022,
    "works_count": 3,
    "cited_by_count": 9
  },
  {
    "year": 2024,
    "works_count": 1,
    "cited_by_count": 1
  },
  {
    "year": 2025,
    "works_count": 1,
    "cited_by_count": 0
  }
]


Now I will add a column to the googleScholar_df called the same as in OpenAlex retrieved dataframe  
  
*orcid_scopus_df["counts_by_year"]*

In [234]:
googleScholar_df["counts_by_year"] = googleScholar_df["full_articles"].apply(
    lambda x: extract_counts_by_year(x) if isinstance(x, list) else []
)

In [238]:
googleScholar_df["counts_by_year"].iloc[0]
#googleScholar_df["counts_by_year"].iloc[1]


[{'year': 2020, 'works_count': 2, 'cited_by_count': 32},
 {'year': 2021, 'works_count': 2, 'cited_by_count': 3},
 {'year': 2022, 'works_count': 3, 'cited_by_count': 9},
 {'year': 2024, 'works_count': 1, 'cited_by_count': 1},
 {'year': 2025, 'works_count': 1, 'cited_by_count': 0}]

#### Internal Consistency Check for first row

In [239]:
from collections import Counter
Counter([art.get("year") for art in sample_articles if isinstance(art, dict)])

Counter({'2022': 3, '2020': 2, '2021': 2, '2024': 1, '2025': 1, '': 1})

In [242]:
# apply extract_counts_by_year on full googleScholar_df

googleScholar_df["counts_by_year"] = googleScholar_df["full_articles"].apply(
    lambda x: extract_counts_by_year(x) if isinstance(x, list) else []
)

In [246]:
googleScholar_df

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata,counts_by_year
0,R_7OE9SfkJJ6q9jbP,https://scholar.google.com/citations?user=BSke...,BSkeONEAAAAJ,{'search_metadata': {'id': '68703f17874d50617c...,0.0,NaN,NaN,NaN,[{'title': 'GDPR Interference With Next Genera...,"{'table': [{'citations': {'all': 45, 'since_20...","[{'year': 2020, 'citations': 3}, {'year': 2021...","{'name': 'Stavroula Rizou, Ph.D.', 'affiliatio...","[{'year': 2020, 'works_count': 2, 'cited_by_co..."
1,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...,None,None,2.0,1.0,1.0,187.0,[{'title': 'Employability and the psychologica...,"{'table': [{'citations': {'all': 2432, 'since_...","[{'year': 1996, 'citations': 6}, {'year': 1997...","{'name': 'Piotr Jędrzejowicz', 'affiliations':...","[{'year': 1959, 'works_count': 1, 'cited_by_co..."
2,R_2B4eUokwACUsT08,https://scholar.google.com/citations?user=DCWk...,DCWkn9sAAAAJ,{'search_metadata': {'id': '68703f19abe7eca231...,1.0,1.0,NaN,118.0,[{'title': 'Ecological impact of introduced fi...,"{'table': [{'citations': {'all': 1169, 'since_...","[{'year': 2011, 'citations': 7}, {'year': 2012...","{'name': 'Rocco Tiberti', 'affiliations': 'DiB...","[{'year': 2006, 'works_count': 1, 'cited_by_co..."
3,R_2JTY94aJonph8Qx,https://scholar.google.it/citations?user=Uyrfi...,UyrfibgAAAAJ,{'search_metadata': {'id': '68703f1b95e95031e3...,1.0,1.0,NaN,118.0,[{'title': 'Agent based modeling and simulatio...,"{'table': [{'citations': {'all': 4117, 'since_...","[{'year': 2004, 'citations': 28}, {'year': 200...","{'name': 'Giuseppe Vizzari', 'affiliations': '...","[{'year': 2002, 'works_count': 2, 'cited_by_co..."
4,R_8S1zWLINCbSqU7z,https://scholar.google.com/citations?user=jqBv...,jqBvlqkAAAAJ,{'search_metadata': {'id': '68703f1e6f1e407892...,2.0,3.0,3.0,188.0,[{'title': 'Injectable and biodegradable hydro...,"{'table': [{'citations': {'all': 7777, 'since_...","[{'year': 2006, 'citations': 25}, {'year': 200...","{'name': 'João Rodrigues', 'affiliations': 'CQ...","[{'year': 1994, 'works_count': 2, 'cited_by_co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,R_8oUd1CmnYTMewcw,https://scholar.google.com/citations?user=Q4_A...,Q4_AHDYAAAAJ,{'error': 'Your account has run out of searche...,1.0,1.0,NaN,222.0,"[{'title': 'Combinatorial bandits revisited', ...","{'table': [{'citations': {'all': 1151, 'since_...","[{'year': 2007, 'citations': 11}, {'year': 200...","{'name': 'M. Sadegh Talebi', 'affiliations': '...","[{'year': 2006, 'works_count': 3, 'cited_by_co..."
841,R_8PXnvStEzBAgC8m,https://scholar.google.com/citations?user=AMLe...,AMLeN5MAAAAJ,{'error': 'Your account has run out of searche...,0.0,NaN,NaN,NaN,[{'title': 'TÜKETİCİLERİN DAVRANIŞLARINDA ŞEKİ...,"{'table': [{'citations': {'all': 139, 'since_2...","[{'year': 2015, 'citations': 2}, {'year': 2016...","{'name': 'ilgar seyidov', 'affiliations': 'Ati...","[{'year': 2013, 'works_count': 2, 'cited_by_co..."
842,R_2sTEG4FLmmU8kOm,https://www.webofscience.com/wos/woscc/basic-s...,None,None,3.0,NaN,NaN,166.0,[],None,None,None,[]
843,R_2w60Oo6hzaQimqC,https://scholar.google.co.uk/citations?user=Wi...,WiW_7VkAAAAJ,{'error': 'Your account has run out of searche...,0.0,NaN,NaN,NaN,"[{'title': 'A random forest guided tour', 'lin...","{'table': [{'citations': {'all': 12950, 'since...","[{'year': 2006, 'citations': 50}, {'year': 200...","{'name': 'Gérard Biau', 'affiliations': 'Profe...","[{'year': 1985, 'works_count': 1, 'cited_by_co..."


In [248]:
empty_counts_mask = googleScholar_df['counts_by_year'].apply(lambda x: isinstance(x, list) and len(x) == 0)
print(f"Number of rows with empty list in counts_by_year: {empty_counts_mask.sum()}")
display(googleScholar_df[empty_counts_mask])


Number of rows with empty list in counts_by_year: 30


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata,counts_by_year
22,R_2gESUa3jSIlreut,https://scholar.google.com/scholar?hl=pl&as_sd...,None,None,1.0,3.0,NaN,187.0,None,None,None,None,[]
132,R_8IuhasFPlu8kYYV,https://scholar.google.com/scholar?hl=no&as_sd...,None,None,2.0,2.0,NaN,82.0,None,None,None,None,[]
204,R_27e32EDeDlr8eyU,https://scholar.google.com/,None,None,2.0,3.0,3.0,243.0,None,None,None,None,[]
224,R_2Eau6BBwERmShZW,https://orcid.org/0000-0002-9925-2173,None,None,0.0,NaN,NaN,NaN,None,None,None,None,[]
259,R_82ms9cdQFpWuwkZ,https://www.bwi.uni-stuttgart.de/institut/team...,None,None,1.0,NaN,NaN,90.0,None,None,None,None,[]
282,R_2EnQS3mqO9PZICY,https://scholar.google.com/scholar?hl=de&as_sd...,None,None,3.0,1.0,1.0,90.0,None,None,None,None,[]
374,R_8bIVRT6jaNhc7Ox,4,None,None,3.0,NaN,NaN,90.0,None,None,None,None,[]
395,R_2WBbA5lTecRmhko,https://scholar.google.com/,None,None,3.0,2.0,2.0,82.0,None,None,None,None,[]
412,R_2UhWPd29jTZYebP,Marcus Clauss,None,None,3.0,NaN,NaN,90.0,None,None,None,None,[]
467,R_82omyzLPNQrS0IG,https://ncl.instructure.com/courses/52235/page...,None,None,2.0,2.0,2.0,243.0,None,None,None,None,[]


In [249]:
empty_counts_response_ids = set(googleScholar_df[empty_counts_mask]['ResponseId'])
unretrieved_response_ids = set(unretrieved_df['ResponseId'])

intersection = empty_counts_response_ids.intersection(unretrieved_response_ids)
print(f"Number of overlapping ResponseIds: {len(intersection)}")
print("Overlapping ResponseIds:", intersection)

Number of overlapping ResponseIds: 0
Overlapping ResponseIds: set()


In [ ]:
# Load the old unretrieved file
unretrieved_df = pd.read_json(
    "not_found_data/unretrieved_GoogleScholar_accounts.json",
    lines=True
)

In [ ]:
# Append the new missing rows
extra_missing_df = googleScholar_df[empty_counts_mask][['ResponseId']]
unretrieved_df = pd.concat([unretrieved_df, extra_missing_df], ignore_index=True).drop_duplicates(subset='ResponseId')

In [255]:
len(unretrieved_df)

113

In [ ]:
# update file 83 + 30 = 113

unretrieved_df.to_json(
    #"not_found_data/unretrieved_GoogleScholar_accounts.json",
    lines=True,
    orient="records"
)


In [258]:
# remove the 30 entries from googleScholar_df

googleScholar_df = googleScholar_df[~googleScholar_df['ResponseId'].isin(empty_counts_response_ids)]


In [ ]:
# recheck whether there are still any rows with empty counts_by_year

empty_counts_mask = googleScholar_df['counts_by_year'].apply(lambda x: isinstance(x, list) and len(x) == 0)
print(f"Number of rows with empty list in counts_by_year: {empty_counts_mask.sum()}")
display(googleScholar_df[empty_counts_mask])


Number of rows with empty list in counts_by_year: 0


,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata,counts_by_year


In [273]:
googleScholar_df

,ResponseId,DE13_1_1,user_id,raw_author_data,DE22,PL6_1,PL6_2,DE23_1_2,full_articles,cited_by,citation_graph,author_metadata,counts_by_year
0,R_7OE9SfkJJ6q9jbP,https://scholar.google.com/citations?user=BSke...,BSkeONEAAAAJ,{'search_metadata': {'id': '68703f17874d50617c...,0.0,NaN,NaN,NaN,[{'title': 'GDPR Interference With Next Genera...,"{'table': [{'citations': {'all': 45, 'since_20...","[{'year': 2020, 'citations': 3}, {'year': 2021...","{'name': 'Stavroula Rizou, Ph.D.', 'affiliatio...","[{'year': 2020, 'works_count': 2, 'cited_by_co..."
1,R_62ojVyfY2SdzKN0,https://scholar.google.com/scholar?hl=en&as_sd...,None,None,2.0,1.0,1.0,187.0,[{'title': 'Employability and the psychologica...,"{'table': [{'citations': {'all': 2432, 'since_...","[{'year': 1996, 'citations': 6}, {'year': 1997...","{'name': 'Piotr Jędrzejowicz', 'affiliations':...","[{'year': 1959, 'works_count': 1, 'cited_by_co..."
2,R_2B4eUokwACUsT08,https://scholar.google.com/citations?user=DCWk...,DCWkn9sAAAAJ,{'search_metadata': {'id': '68703f19abe7eca231...,1.0,1.0,NaN,118.0,[{'title': 'Ecological impact of introduced fi...,"{'table': [{'citations': {'all': 1169, 'since_...","[{'year': 2011, 'citations': 7}, {'year': 2012...","{'name': 'Rocco Tiberti', 'affiliations': 'DiB...","[{'year': 2006, 'works_count': 1, 'cited_by_co..."
3,R_2JTY94aJonph8Qx,https://scholar.google.it/citations?user=Uyrfi...,UyrfibgAAAAJ,{'search_metadata': {'id': '68703f1b95e95031e3...,1.0,1.0,NaN,118.0,[{'title': 'Agent based modeling and simulatio...,"{'table': [{'citations': {'all': 4117, 'since_...","[{'year': 2004, 'citations': 28}, {'year': 200...","{'name': 'Giuseppe Vizzari', 'affiliations': '...","[{'year': 2002, 'works_count': 2, 'cited_by_co..."
4,R_8S1zWLINCbSqU7z,https://scholar.google.com/citations?user=jqBv...,jqBvlqkAAAAJ,{'search_metadata': {'id': '68703f1e6f1e407892...,2.0,3.0,3.0,188.0,[{'title': 'Injectable and biodegradable hydro...,"{'table': [{'citations': {'all': 7777, 'since_...","[{'year': 2006, 'citations': 25}, {'year': 200...","{'name': 'João Rodrigues', 'affiliations': 'CQ...","[{'year': 1994, 'works_count': 2, 'cited_by_co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,R_2jxnOFfTG7Rh0qJ,https://scholar.google.com/citations?hl=en&aut...,0BT2XcAAAAAJ,{'error': 'Your account has run out of searche...,0.0,NaN,NaN,NaN,[{'title': 'Influence of the COVID-19 lockdown...,"{'table': [{'citations': {'all': 143, 'since_2...","[{'year': 2019, 'citations': 3}, {'year': 2020...","{'name': 'Fabio Parente', 'affiliations': 'Lec...","[{'year': 2016, 'works_count': 1, 'cited_by_co..."
840,R_8oUd1CmnYTMewcw,https://scholar.google.com/citations?user=Q4_A...,Q4_AHDYAAAAJ,{'error': 'Your account has run out of searche...,1.0,1.0,NaN,222.0,"[{'title': 'Combinatorial bandits revisited', ...","{'table': [{'citations': {'all': 1151, 'since_...","[{'year': 2007, 'citations': 11}, {'year': 200...","{'name': 'M. Sadegh Talebi', 'affiliations': '...","[{'year': 2006, 'works_count': 3, 'cited_by_co..."
841,R_8PXnvStEzBAgC8m,https://scholar.google.com/citations?user=AMLe...,AMLeN5MAAAAJ,{'error': 'Your account has run out of searche...,0.0,NaN,NaN,NaN,[{'title': 'TÜKETİCİLERİN DAVRANIŞLARINDA ŞEKİ...,"{'table': [{'citations': {'all': 139, 'since_2...","[{'year': 2015, 'citations': 2}, {'year': 2016...","{'name': 'ilgar seyidov', 'affiliations': 'Ati...","[{'year': 2013, 'works_count': 2, 'cited_by_co..."
843,R_2w60Oo6hzaQimqC,https://scholar.google.co.uk/citations?user=Wi...,WiW_7VkAAAAJ,{'error': 'Your account has run out of searche...,0.0,NaN,NaN,NaN,"[{'title': 'A random forest guided tour', 'lin...","{'table': [{'citations': {'all': 12950, 'since...","[{'year': 2006, 'citations': 50}, {'year': 200...","{'name': 'Gérard Biau', 'affiliations': 'Profe...","[{'year': 1985, 'works_count': 1, 'cited_by_co..."


In [275]:
googleScholar_df.to_json("SerpAPI_scrapes/unified_dataset/GoogleScholar_Pubhistory.json", lines=True, orient="records")

### Assessing Data Integrity

In [261]:
def compute_article_integrity(articles):
    year_counts = defaultdict(lambda: {"works": 0, "cites": 0})
    integrity = {
        "total_articles": 0,
        "non_dict_entries": 0,
        "missing_years": 0,
        "missing_citations": 0,
        "bad_year_keys": []
    }

    for art in articles:
        integrity["total_articles"] += 1

        if not isinstance(art, dict):
            integrity["non_dict_entries"] += 1
            continue

        y = art.get("year")
        c = art.get("cited_by", {}).get("value", 0)

        # Check for bad year
        if not isinstance(y, int):
            integrity["missing_years"] += 1
            y = ""  # placeholder key — the issue we’re seeing
        else:
            y = str(y)  # normalize year to string for consistency

        if y == "":
            integrity["bad_year_keys"].append(y)

        if not isinstance(c, int):
            integrity["missing_citations"] += 1
            c = 0

        year_counts[y]["works"] += 1
        year_counts[y]["cites"] += c

    return year_counts, integrity


In [ ]:
googleScholar_df["article_integrity"] = googleScholar_df["full_articles"].apply(
    lambda x: compute_article_integrity(x) if isinstance(x, list) else None
)

In [267]:
# Flatten the integrity dict
integrity_fields = [
    'total_articles',
    'non_dict_entries',
    'missing_years',
    'missing_citations',
    'bad_year_keys'
]

for field in integrity_fields:
    googleScholar_df[field] = googleScholar_df["article_integrity"].apply(
        lambda x: x[1].get(field) if isinstance(x, tuple) else None
    )


In [270]:
len(googleScholar_df[googleScholar_df["missing_years"] > 0])


815

In [269]:
for i, row in googleScholar_df[googleScholar_df["missing_years"] > 0].head(5).iterrows():
    print(f"Index: {i}")
    for art in row["full_articles"]:
        print(art.get("year"), art.get("title"))
    print("\n---\n")


Index: 0
2020 GDPR Interference With Next Generation 5G and IoT Networks
2022 Preserving Minors' Data Protection in IoT-based Smart Homes According to GDPR Considering Cross-Border Issues.
2022 COVID-19 Impacts in the New Technological Era: Cross-Border Privacy Issues With Emphasis on AI
2021 Protecting Minors’ Personal Data in IoT-based Smart Homes According to GDPR
2020 Taxonomy about the Stages of Performing Automated Decision-Making Processing under GDPR in the Light of 6G Networks
2024 Training in Co-Creation as a Methodological Approach to Improve AI Fairness
2021 Enhanced Privacy Recommendations According to GDPR in the Context of Internet-of-Things (IoT)
2025 Achieving Socio-Economic Parity through the Lens of EU AI Act
2022 Διασυνοριακή ροή οικονομικών δεδομένων: νομική προσέγγιση
 AFTER COVID-19: CRISES, ETHICS, AND SOCIO-TECHNICAL CHANGE

---

Index: 1
2008 Employability and the psychological contract in European ICT sector SMEs
2016 Employability and job performance as link